In [1]:
from IPython.display import display, HTML

full_parchment_style_with_headings = """
<style>
  /* Overall font and background */
  body, .notebook-app, .container, .cell, .text_cell_render, .output_area {
    font-family: 'Georgia', 'Palatino Linotype', Palatino, serif !important;
    font-size: 15px !important;
    line-height: 1.7 !important;
    color: #3b3a32 !important;
    background-color: #f9f6f1 !important; /* soft parchment */
  }

  /* Container */
  .notes-container {
    margin: 2em auto;
    max-width: 900px;
  }

  /* Cards styled like parchment */
  .note-card {
    background: #fbf8f1; /* very pale parchment */
    border: 1px solid #d3c9b7;
    border-radius: 14px;
    padding: 2em 2.5em;
    margin-bottom: 2.5em;
    box-shadow: 0 3px 10px rgba(115,100,81,0.1);
    transition: box-shadow 0.3s ease;
  }
  .note-card:hover {
    box-shadow: 0 7px 22px rgba(115,100,81,0.15);
  }

  /* Global headings outside cards */
  h1 {
    font-family: 'Georgia', serif;
    font-size: 2.8em;
    font-weight: 700;
    color: #4b4636; /* dark olive-brown */
    margin-top: 0;
    margin-bottom: 0.8em;
    border-bottom: 3px solid #c1bfae; /* soft beige underline */
    padding-bottom: 0.4em;
  }

  h2 {
    font-family: 'Georgia', serif;
    font-size: 2.1em;
    font-weight: 600;
    color: #5a533d; /* muted olive-green */
    margin-top: 0;
    margin-bottom: 0.7em;
    border-bottom: 2px solid #d3c9b7; /* lighter beige underline */
    padding-bottom: 0.3em;
  }

  /* Headings inside cards use h3 */
  .note-card h3 {
    font-family: 'Georgia', serif;
    font-size: 1.6em;
    font-weight: 600;
    color: #5a533d; /* muted olive-green */
    margin-top: 0;
    margin-bottom: 0.75em;
    border-bottom: 2px solid #d3c9b7;
    padding-bottom: 0.25em;
  }

  /* Paragraph text */
  .note-card p {
    color: #4c4b44;
    font-size: 15px;
    margin-bottom: 1.3em;
  }

  /* Lists */
  .note-card ul {
    list-style-type: disc;
    margin-left: 1.5em;
    padding-left: 0.5em;
  }
  .note-card li {
    margin-bottom: 0.75em;
    color: #56534b;
    font-size: 15px;
  }

  /* Math emphasis block */
  .math-center {
    text-align: center;
    font-size: 14.5px;
    color: #706a57;
    background: #f0ede3; /* warm parchment */
    padding: 1em 1.5em;
    border-radius: 12px;
    margin-top: 1.4em;
    margin-bottom: 1.8em;
    font-style: italic;
    box-shadow: inset 0 0 8px rgba(115,100,81,0.1);
  }

  /* Info callout */
  .info {
    display: inline-block;
    background-color: #e4decf; /* soft warm beige */
    color: #625d4f;
    font-weight: 600;
    padding: 0.4em 0.8em;
    border-radius: 7px;
    box-shadow: 0 1px 4px rgba(115,100,81,0.12);
  }
</style>
"""


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, average_precision_score
from sklearn.datasets import make_classification

sns.set_palette('Spectral')

/home/mustafa/miniconda3/envs/t2/lib/python3.13/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [3]:
import numpy as np
import matplotlib.pyplot as plt

class CustomLogisticRegression:
    def __init__(self, learning_rate=0.01, number_of_epochs=1000,
                 verbose=False, log_every=100):
        self.weights_vector = None            # weights of features (coefficients). shape(n_features,)
        self.bias = None                      # bias (intercept)
        
        self.learning_rate = learning_rate    # learning rate
        self.number_of_epochs = number_of_epochs  # number of training epochs
        
        self.final_loss = None                # last loss value of the model
        self.loss_history = []                # all losses generated by the model
        
        self.verbose = verbose                # print epoch | cost | weights norm
        self.log_every = log_every            # log how often 

    def _init_parameters(self, n_features):
        """Initialize weights and bias."""
        self.weights_vector = np.zeros(n_features)
        self.bias = 0.0

    def _sigmoid(self, z):
        """Sigmoid activation function."""
        z = np.clip(z, -500, 500)  # avoid overflow
        return 1 / (1 + np.exp(-z))
    
    def _log_loss(self, y, y_hat):
        """Compute the logistic loss."""
        epsilon = 1e-10
        return -np.mean(y * np.log(y_hat + epsilon) + (1 - y) * np.log(1 - y_hat + epsilon))
    
    def _forward_propagation(self, X, y):
        """Compute predictions and loss."""
        z = X @ self.weights_vector + self.bias
        y_hat = self._sigmoid(z)
        loss = self._log_loss(y, y_hat)
        self.loss_history.append(loss)
        return y_hat, loss

    def _back_propagation(self, X, y, y_hat):
        """Compute gradients."""
        error = y_hat - y
        n_samples = X.shape[0]
        weights_derivative = (X.T @ error) / n_samples
        bias_derivative = np.mean(error)
        return weights_derivative, bias_derivative

    def _update(self, weights_derivative, bias_derivative):
        """Update weights and bias."""
        self.weights_vector -= self.learning_rate * weights_derivative
        self.bias -= self.learning_rate * bias_derivative
        
    def fit(self, X, y):
        """Train the logistic regression model."""
        self._init_parameters(X.shape[1])

        for epoch in range(self.number_of_epochs):
            y_hat, current_loss = self._forward_propagation(X, y)
            weights_derivative, bias_derivative = self._back_propagation(X, y, y_hat)
            self._update(weights_derivative, bias_derivative)

            if self.verbose and (epoch % self.log_every == 0 or epoch == self.number_of_epochs - 1):
                print(f"Epoch {epoch:<5} | Loss: {current_loss:.5f} | Weights Norm: {np.linalg.norm(self.weights_vector):.5f}")

        self.final_loss = current_loss
        return self
    
    def predict_probability(self, X):
        """Predict probabilities for input samples."""
        z = X @ self.weights_vector + self.bias
        return self._sigmoid(z)

    def predict(self, X):
        """Predict class labels (0 or 1)."""
        probabilities = self.predict_probability(X)
        return (probabilities >= 0.5).astype(int)

    def plot_losses(self, ax=None):
        """Plot losses over epochs."""
        if ax is None:
            ax = plt.gca()
        ax.plot(self.loss_history, label=f'Final loss: {self.final_loss:.2f}')
        ax.set_xlabel('Epochs', fontsize=12)
        ax.set_ylabel('Loss', fontsize=12)
        ax.set_title('Training Loss Over Epochs')
        ax.legend()
        ax.grid(alpha=0.5)
